# Logistic regression

Filippo Calzavara

Nicolae Righeriu

## Imports

In [1]:
import math
import random
import pyspark
from itertools import chain
import datetime
import os
import time
from IPython.core.display import display, HTML
display(HTML("<style>.container { width:100% !important; }</style>"))

global k_fold
global n_row
global y_h_m_y
global n_columns
global w
global OUTPUT
global ITERATIONS
global THRESHOLD
global averages
global col_sums
global sigmas
global b

DEBUG = True
TRAIN_TEST = 0.8
k_fold=3
ITERATIONS=50
THRESHOLD = 0.5

DATASET="dataset/spam.data"
n_columns=56 #total number of columns (58) - the one deleted (57) - label (56)

OUTPUT="out/"

## Initialization

In [2]:
def initializeAccumulators():
    global averages
    global col_sums
    global sigmas
    i=0
    averages=[]
    col_sums=[]
    sigmas=[]
    while(i<n_columns):
        averages.append(sc.accumulator(0))
        col_sums.append(sc.accumulator(0))
        sigmas.append(sc.accumulator(0)) 
        i+=1

def initializeWeights(random_init=False):
    if(random_init):
        #return sc.parallelize([(i, random.random()) for i in range(0,n_columns)])
        return [random.random() for i in range(0,n_columns)]
    else:
        #return sc.parallelize([(i, 0.0) for i in range(0,n_columns)])
        return [0.0 for i in range(0,n_columns)]

def initializeBias():
    return sc.parallelize(0.0)


## Preprocessing

### Metrics calculation for normalization

In [3]:
def addToAccumulators(row):
    if(len(row)!=len(col_sums)):
        raise Exception("Number of columns in the row doesn't mach the number of accumulators initiated. Len row: "+str(len(row))+" n_accomulators: "+str(len(col_sums)))
    i=0
    while(i<n_columns):
        col_sums[i].add(row[i])
        i+=1

def calcAvg(n_row):
    global averages
    i=0
    while(i<n_columns):
        averages[i]=col_sums[i].value/n_row
        i+=1

def calcResiduals(row):
    global sigmas
    i=0
    while(i<n_columns):       
        sigmas[i].add(math.pow(row[2][i]-averages[i],2))
        i+=1
    return row

def calcSigmas(n_row):
    global sigmas
    i=0
    while(i<n_columns):
        sigmas[i]=math.sqrt(sigmas[i].value/float(n_row-1))
        i+=1

In [4]:
def normalize(row):
    i=0
    while(i<n_columns):
        row[2][i]=(row[2][i]-averages[i])/sigmas[i]
        i+=1
    return row

### Preprocessing

In [ ]:
def preprocessing(row):
    split=row.split(" ")       #splitting
    label = int(split[57])
    del split[57] #label
    del split[56] #col 57th
    split=[float(col) for col in split]
    addToAccumulators(split)
    #assign random key to the datapoint
    key=random.getrandbits(64)
    #assign train/test 
    if(random.random()<TRAIN_TEST):
        train=1
    else:
        train=0    
    return (key,train,split,label)

### Split

In [ ]:
def trainTestSplit(dataset):
    dataset=dataset.map(normalize)
    train=dataset.filter(lambda x: x[1]==1)
    test=dataset.filter(lambda x: x[1]==0)
    return train, test

## Prediction functions and cost

In [ ]:
def sigmoid(z):
    return 1/(1+math.exp(-z))

def predict(w,b,X):
    sig=sigmoid(sum([X[i]*w[i] for i in range(len(w))])+b)
    return sig

def classify_prediction(pred_probability):
    if (pred_probability >= THRESHOLD):
        return 1
    return 0

def predict_parallel(w,b,X):
    #Change X representation
    X = sc.parallelize(X).zipWithIndex().map(lambda x: (x[1],x[0]))
    wX_plus_b=X.join(w).map(lambda x: (x[1][0]*x[1][1])+b).sum()
    return sigmoid(wX_plus_b)

def compute_cost(dataset,w,b,lambda_reg, print_stats=False):
    cost=0
    #print("predict(w,b,x[2]))="+str(predict(w,b,dataset.collect()[0][2])))
    #print("math.log(predict(w,b,x[2]))="+str(math.log(predict(w,b,dataset.collect()[0][2]))))
    cost = (-1/dataset.count())*dataset.map(lambda x: x[3]*math.log(predict(w,b,x[2]))+(1-x[3])*math.log(1-predict(w,b,x[2]))).sum()+lambda_reg/(2*dataset.count())*sum([i*i for i in w])
    
    '''
    print("RowNumber: "+str(dataset.count()))
    print("\nlambda_reg: "+str(lambda_reg))
    print("\nb: "+str(b))
    print("\nw: "+str(w))
    print("\nDataset: "+str(w))
    '''
    if (print_stats):
        stats = {"TP":0, "TN":0, "FP":0, "FN":0}
        statsMapping = dataset.map(lambda x: 2*(classify_prediction(predict(w, b, x[2])) - x[3]) + x[3])  #this function maps all 4 possible variations to an integer, from -1 to 3
        stats["TP"] = statsMapping.filter(lambda x: x == 1).count()
        stats["TN"] = statsMapping.filter(lambda x: x == 2).count()
        stats["FP"] = statsMapping.filter(lambda x: x == -1).count()
        stats["FN"] = statsMapping.filter(lambda x: x == 0).count()
        precision = stats["TP"]/(stats["TP"] + stats["FP"])
        recall = stats["TP"]/(stats["TP"] + stats["FN"])
        accuracy = (stats["TP"] + stats["TN"])/(stats["TP"] + stats["TN"] + stats["FP"] + stats["FN"])
        print("Precision: " + str(precision))
        print("Recall: " + str(recall))
        print("F1 score: " + str(2*(precision*recall)/(precision + recall)))
        print("Accuracy: " + str(accuracy))
    return cost

def testParameters(test, w, b):
    test_error = test.map(lambda x: ())

## Gradient Descent

In [ ]:
def gradientDescent(iterations,train,w,number_samples,lambda_reg,learning_rate,b,logCosts=False):
    if(logCosts):
        gd_cost_log = open(OUTPUT+"gradient_descent.csv","w+")
        gd_cost_log.write("iteration,cost\r\n")
        gd_cost_log.close()
    costs=[]
    for iteration in range(iterations):
        w,b = gradientDescentIteration(train,w,number_samples,lambda_reg,learning_rate,b)        
        cost=compute_cost(train,w,b,lambda_reg)
        costs.append(cost)
        if(logCosts):
            gd_cost_log = open(OUTPUT+"gradient_descent.csv","a+")            
            gd_cost_log.write(str(iteration+1)+","+str(cost)+"\r\n")
            gd_cost_log.close()
        print("-> Iteration done: "+str(iteration+1)+" of "+str(iterations)+". Cost: "+str(cost))
    return w,b,costs
    
def gradientDescentIteration(train,w,number_samples,lambda_reg,learning_rate,b):
    dw=[0 for i in range(0,n_columns)]
    j = 0
    w_temp = w.copy()
    for j in range(n_columns):
        X_j=train.map(lambda x: (predict(w,b,x[2])-x[3])*x[2][j]).sum()
        dw[j]=(1/number_samples)*X_j+(lambda_reg/number_samples)+w[j]
        w_temp[j]-=learning_rate*dw[j]
    b-=learning_rate*(1/number_samples)*train.map(lambda x: predict(w,b,x[2])-x[3]).sum()
    w = w_temp
    return w_temp,b

## K-Fold Cross Validation

In [ ]:
def make_folds(x):
    return (x[0],int(random.random()*10-1),x[1],x[2])

def kFoldsCV(k_fold,iterations,train,lambda_reg,learning_rate):
    fold_length=train.count()/k_fold
    train_errors_fold=[]
    test_errors_fold = []
    for i_fold in range(k_fold):
        w=initializeWeights()
        b=0
        starting_fold=fold_length*i_fold
        end_fold=starting_fold+fold_length
        test_fold=train.zipWithIndex().filter(lambda t: (t[1]>=starting_fold and t[1]<end_fold)).map(lambda t: t[0]) #the map get rid of the index again
        train_fold=train.zipWithIndex().filter(lambda t: t[1]<starting_fold or t[1]>=end_fold).map(lambda t: t[0])
        train_fold_size=train_fold.count()
        
        #Gradient descent
        w, b, train_errors = gradientDescent(iterations,train_fold,w,train_fold_size,lambda_reg,learning_rate,b)
        train_errors_fold.append(train_errors)
        train_errors_flattened =list(chain.from_iterable(train_errors_fold))
        test_error = compute_cost(test_fold, w, b, lambda_reg)
        test_errors_fold.append(test_error)
        if(DEBUG):
            print("--> Fold #"+str(i_fold+1)+" of "+str(k_fold)+" is done. Train error: " \
                  +str(sum(train_errors_flattened)/((i_fold + 1) * iterations)) \
                  + " Test error: " + str(sum(test_errors_fold)/(i_fold+ 1)))
    return w,b,train_errors_fold, test_errors_fold

## Train

In [ ]:
def train(filename, iterations, learning_rate, lambda_reg, cv=False):
    global w
    global b
    w = []
    initializeAccumulators()
    w = initializeWeights()
    dataset=sc.textFile(filename).map(preprocessing)
    n_row=dataset.count()
    calcAvg(n_row)
    gg = dataset.map(calcResiduals).collect()
    dataset=dataset.collect()
    calcSigmas(n_row)
    dataset=sc.parallelize(dataset).sortBy(lambda x: x[0])
    n_row=dataset.count()
    train, test = trainTestSplit(dataset)
    
    print("Split train/test done. Train contains "+str(train.count())+" elements, Test contains "+str(test.count())+" elements")
    
    if (cv):
        w, b, train_errors_fold, test_errors_fold = kFoldsCV(k_fold,iterations,train,lambda_reg,learning_rate)
        train_errors_fold =list(chain.from_iterable(train_errors_fold))
        average_train_error = sum(train_errors_fold)/(iterations*k_fold)
        average_test_error = sum(test_errors_fold)/k_fold
        print("---> "+str(k_fold)+"-fold validation done.  Train error: " \
            +str(average_train_error) \
            + " Test error: " + str((average_test_error)))
        return average_train_error, average_test_error
    else:                                                  #Testing the parameters and weights on the actual test set
        w,b, train_errors = gradientDescent(iterations, train, w, train.count(), lambda_reg, learning_rate, b,True)
        wBest = open(OUTPUT+"w_bestModel.csv","w+")
        wBest.write(str(w))
        wBest.close()
        average_train_error = sum(train_errors)/iterations
        average_test_error = compute_cost(test, w, b, lambda_reg, True)
        print("---> Model done. Train error: " \
              +str(average_train_error) \
              + " Test error: " + str((average_test_error)))
        return average_train_error, average_test_error

## Main

### Logging

In [ ]:
def touch(path):
    with open(path, 'w+'):
        os.utime(path, None)
workers = range(1,9) #8 cores
working_time = open(OUTPUT+"working_times.csv","w+")
#working_time.write("workers,grid_time,best_model_time,total\r\n")
working_time.write("workers,grid_time,total\r\n")
working_time.close()
grid_log = open(OUTPUT+"grid.csv","w+")
grid_log.write("learning_rate,lambda_red,train_error,test_error\r\n")
grid_log.close()

### Workers and grid

In [ ]:
for worker in workers:
    global b
    working_time = open(OUTPUT+"working_times.csv","a+")
    print("****Worker: "+str(worker)+"****\n")
    sc = pyspark.SparkContext.getOrCreate()
    sc.stop()
    conf = (SparkConf().set("spark.cores.max", str(worker)))
    sc = SparkContext(conf=conf)
    b = 0
    col_sums=[]
    averages=[]
    sigmas=[]
    
    w=[]
    if (False):
        grid=[]
        for i in range(1,5):
                for j in range(1,14,3):
                    grid+=[(i*0.09,j*0.028)]
    
        #grid=[(0.01,)] #(learning_rate,lambda_reg)
        #grid=list(chain.from_iterable(grid))
        grid_results = []
        #workers_executioTime_grid=time.time()
        for par in grid:
            print("|---- Starting training for parameters (learning_rate,lambda_reg) = "+str(par)+" ----|")
            result=(par, train(DATASET,ITERATIONS,par[0],par[1], True))
            grid_results.append(result)
            grid_log = open(OUTPUT+"grid.csv","a+")
            grid_log.write(str(result[0][0])+","+str(result[0][1])+","+str(result[1][0])+","+str(result[1][1])+"\r\n")
            grid_log.close()
        #workers_executioTime_grid=time.time()-workers_executioTime_grid
        
        best_par = sorted(grid_results, key=lambda x: x[1])[0]
    workers_executioTime_bestTraining= time.time()
    best_par = (0.2, 0.05)
    print("\n--------------------------------------------------")
    print("Best parameters: " + str(best_par))
    smallest_error = train(DATASET,ITERATIONS,best_par[0],best_par[1])
    print("Best performance: " + str(smallest_error))
    print("--------------------------------------------------")
    #print(w)
    workers_executioTime_bestTraining=time.time()-workers_executioTime_bestTraining
    #working_time.write(str(worker)+","+str(workers_executioTime_grid)+","+str(workers_executioTime_bestTraining)+","+str(workers_executioTime_grid+workers_executioTime_bestTraining)+"\r\n")
    working_time.write(str(worker)+","+str(workers_executioTime_bestTraining)+"\r\n")
    working_time.close()
    #print("++++Worker "+str(worker)+" has terminated with running time: "+str(workers_executioTime_grid+workers_executioTime_bestTraining)+"++++\n")
    print("++++Worker "+str(worker)+" has terminated with running time: "+str(workers_executioTime_bestTraining)+"++++\n")

    

****Worker: 1****


--------------------------------------------------
Best parameters: (0.2, 0.05)
Split train/test done. Train contains 3636 elements, Test contains 965 elements
-> Iteration done: 1 of 50. Cost: 0.6139579036819058
-> Iteration done: 2 of 50. Cost: 0.5725468093431973
-> Iteration done: 3 of 50. Cost: 0.5479742078763638
-> Iteration done: 4 of 50. Cost: 0.5323568066258672
-> Iteration done: 5 of 50. Cost: 0.5219600413297797
-> Iteration done: 6 of 50. Cost: 0.514791831486005
-> Iteration done: 7 of 50. Cost: 0.5097035069217286
-> Iteration done: 8 of 50. Cost: 0.505995773941176
-> Iteration done: 9 of 50. Cost: 0.5032261700821264
-> Iteration done: 10 of 50. Cost: 0.5011067966737472
-> Iteration done: 11 of 50. Cost: 0.49944635887442007
-> Iteration done: 12 of 50. Cost: 0.4981156354293895
-> Iteration done: 13 of 50. Cost: 0.4970261161419953
-> Iteration done: 14 of 50. Cost: 0.49611640878549296
-> Iteration done: 15 of 50. Cost: 0.49534340720992
-> Iteration done: 16